In [1]:
import random

In [2]:
N_JOBS = 7 #CHROMOSOME
N_MACHINES = 4 #GENES

In [3]:
pop_size = 400
generations = 500
mt_rate = 0.1
tm_size = 5

# 1. Generate Data

In [24]:
import random
# The processing time of a job at a machine is defined in a matrix proc[job][machine]
def generateData(seed=0):
    PROC = [ [ 0 for m in range(N_MACHINES) ] for j in range(N_JOBS) ]
    random.seed(seed)
    for j in range(N_JOBS):
        for m in range(N_MACHINES):
            PROC[j][m] = random.randint(1,9)
    return PROC

In [25]:
# Initialisation code base on the last 4 digits of Student ID: 2312 7635
PROC = generateData(7635) 
PROC #Population

[[3, 6, 7, 3],
 [9, 1, 4, 8],
 [3, 5, 1, 3],
 [5, 5, 3, 9],
 [5, 1, 8, 5],
 [5, 9, 9, 3],
 [2, 6, 1, 7]]

# 2. Processing times

In [26]:
def processing_time(job_sequence, PROC):
    total_time, _, _, _, _ = optimal_time(PROC, job_sequence)
    return total_time

# 3. Initial Population

In [27]:
def ini_population(pop_size, N_JOBS):
    population = []
    for _ in range(pop_size):
        individual = list(range(N_JOBS))
        random.shuffle(individual)
        population.append(individual)
    return population

# 4. Selection

In [28]:
def tournament(population, fitness_score, tm_size):
    tournament_indices = random.sample(range(len(population)), tm_size)
    tournament_population = [population[i] for i in tournament_indices]
    tournament_fitness = [fitness_score[i] for i in tournament_indices]
    winner_index = tournament_fitness.index(max(tournament_fitness))
    return tournament_population[winner_index]

# 5. Crossover

In [29]:
def crossover(parent1, parent2):
    child = [-1] * len(parent1)
    start_pos = random.randint(0, len(parent1) - 1)
    end_pos = random.randint(0, len(parent1) - 1)

    if start_pos < end_pos:
        for i in range(start_pos, end_pos + 1):
            child[i] = parent1[i]
    elif start_pos > end_pos:
        for i in range(end_pos, start_pos + 1):
            child[i] = parent1[i]

    for i in range(len(parent2)):
        if parent2[i] not in child:
            for j in range(len(child)):
                if child[j] == -1:
                    child[j] = parent2[i]
                    break
    return child

# 6. Mutation

In [30]:
def mutation(sequence, mutation_rate=0.01):
    for i in range(len(sequence)):
        if random.random() < mutation_rate:
            j = random.randint(0, len(sequence) - 1)
            sequence[i], sequence[j] = sequence[j], sequence[i]

In [31]:
def optimal_time(PROC, job_sequence):
    N_JOBS = len(PROC)
    N_MACHINES = len(PROC[0])
    
    idle_time = [[0] * N_MACHINES for _ in range(N_JOBS)]
    start_time = [[0] * N_MACHINES for _ in range(N_JOBS)]
    all_time = [[0] * N_MACHINES for _ in range(N_JOBS)]
    stop_time = [[0] * N_MACHINES for _ in range(N_JOBS)]
    wait_time = [[0] * N_MACHINES for _ in range(N_JOBS)]

    for job_index in range(N_JOBS):
        job = job_sequence[job_index]
        for machine in range(N_MACHINES):
            if job_index == 0 and machine == 0:
                start_time[job][machine] = 0
            elif job_index == 0:
                start_time[job][machine] = all_time[job][machine - 1]
            elif machine == 0:
                start_time[job][machine] = all_time[job_sequence[job_index - 1]][machine]
            else:
                start_time[job][machine] = max(all_time[job][machine - 1], all_time[job_sequence[job_index - 1]][machine])

            if job_index > 0:
                idle_time[job][machine] = max(0, start_time[job][machine] - all_time[job_sequence[job_index - 1]][machine])

            if machine > 0:
                wait_time[job][machine] = start_time[job][machine] - stop_time[job][machine - 1]

            all_time[job][machine] = start_time[job][machine] + PROC[job][machine]
            stop_time[job][machine] = all_time[job][machine]

    last_job = job_sequence[-1]
    total_time = all_time[last_job][N_MACHINES - 1]

    return total_time, start_time, idle_time, stop_time, wait_time

# 7. Genetic Algorithm

In [32]:
def GA_sequence(PROC, pop_size=100, generations=1000, tm_size=5, mutation_rate=0.01):
    N_JOBS = len(PROC)
    population = ini_population(pop_size, N_JOBS)
    optimal_sequence = None
    optimal_fitness = float('inf')
    
    for generation in range(generations):
        fitness_score = []
        times_list = []
        for seq in population:
            total_time = processing_time(seq, PROC)
            fitness_score.append(-total_time)  # Using negative total time for maximization
            times_list.append(total_time)
        
        new_population = []
        
        for _ in range(pop_size):
            parent1 = tournament(population, fitness_score, tm_size)
            parent2 = tournament(population, fitness_score, tm_size)
            child = crossover(parent1, parent2)
            mutation(child, mutation_rate)
            new_population.append(child)
        
        population = new_population
        new_optimal_fitness = max(fitness_score)
        new_optimal_sequence = population[fitness_score.index(new_optimal_fitness)]
        
        if -new_optimal_fitness < optimal_fitness:
            optimal_fitness = -new_optimal_fitness
            optimal_sequence = new_optimal_sequence
            optimal_times = optimal_time(PROC, optimal_sequence)
        
        print(f"Generation {generation}: Optimal Fitness: {optimal_fitness}"
              f" | Optimal sequence: {optimal_sequence}")
    
    return optimal_sequence, optimal_times

In [45]:
optimal_sequence, optimal_times = GA_sequence(PROC)

Generation 0: Optimal Fitness: 53 | Optimal sequence: [2, 0, 1, 3, 4, 6, 5]
Generation 1: Optimal Fitness: 53 | Optimal sequence: [2, 0, 1, 3, 4, 6, 5]
Generation 2: Optimal Fitness: 53 | Optimal sequence: [2, 0, 1, 3, 4, 6, 5]
Generation 3: Optimal Fitness: 53 | Optimal sequence: [2, 0, 1, 3, 4, 6, 5]
Generation 4: Optimal Fitness: 53 | Optimal sequence: [2, 0, 1, 3, 4, 6, 5]
Generation 5: Optimal Fitness: 53 | Optimal sequence: [2, 0, 1, 3, 4, 6, 5]
Generation 6: Optimal Fitness: 53 | Optimal sequence: [2, 0, 1, 3, 4, 6, 5]
Generation 7: Optimal Fitness: 53 | Optimal sequence: [2, 0, 1, 3, 4, 6, 5]
Generation 8: Optimal Fitness: 53 | Optimal sequence: [2, 0, 1, 3, 4, 6, 5]
Generation 9: Optimal Fitness: 53 | Optimal sequence: [2, 0, 1, 3, 4, 6, 5]
Generation 10: Optimal Fitness: 53 | Optimal sequence: [2, 0, 1, 3, 4, 6, 5]
Generation 11: Optimal Fitness: 53 | Optimal sequence: [2, 0, 1, 3, 4, 6, 5]
Generation 12: Optimal Fitness: 53 | Optimal sequence: [2, 0, 1, 3, 4, 6, 5]
Generatio

In [46]:
total_time, start_time, idle_time, stop_time, wait_time = optimal_times

In [47]:
start_time

[[7, 13, 19, 26],
 [18, 29, 32, 41],
 [10, 19, 26, 29],
 [13, 24, 29, 32],
 [0, 5, 6, 14],
 [27, 32, 41, 50],
 [5, 7, 14, 19]]

In [48]:
idle_time

[[0, 0, 4, 0],
 [0, 0, 0, 0],
 [0, 0, 0, 0],
 [0, 0, 2, 0],
 [0, 0, 0, 0],
 [0, 2, 5, 1],
 [0, 1, 0, 0]]

In [49]:
wait_time

[[0, 3, 0, 0],
 [0, 2, 2, 5],
 [0, 6, 2, 2],
 [0, 6, 0, 0],
 [0, 0, 0, 0],
 [0, 0, 0, 0],
 [0, 0, 1, 4]]

In [50]:
stop_time

[[10, 19, 26, 29],
 [27, 30, 36, 49],
 [13, 24, 27, 32],
 [18, 29, 32, 41],
 [5, 6, 14, 19],
 [32, 41, 50, 53],
 [7, 13, 15, 26]]

# 8. Structure of the schedule

In [51]:
def schedule(optimal_sequence, optimal_times, PROC):
    
   
    row = ""
    row += '|---------|'
    for m in range(N_MACHINES):
        row += f"------------|" \
               f"------------|"
    row += '\n'
    row += '|         |'
    for m in range(N_MACHINES):
        row += f"            |" \
               f" Machine: {m:1d} |"
    row += '\n'
    for j in range(N_JOBS):
        row += '|---------|'
        for m in range(N_MACHINES):
            row += f"------------|" \
                   f"------------|"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | " \
                   f"Idle: {(idle_time[j][m]):4d} |"
        row += '\n'
        row += '|---------|'
        for m in range(N_MACHINES):
            row += f"------------|" \
                   f"------------|"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | " \
                   f"Start:{(start_time[j][m]):4d} |"  
        row += '\n' 
        row += f'| Job: {j+1:2d} |'
        for m in range(N_MACHINES):
            row += f" Wait: {(wait_time[j][m]):4d} | " \
                   f"Proc: {(PROC[j][m]):4d} |"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | " \
                   f"Stop: {(stop_time[j][m]):4d} |"  
        row += '\n' 
    row += '|---------|'
    for m in range(N_MACHINES):
        row += f"------------|" \
               f"------------|"
    row += '\n'
    return row

The Optimal Job Schedule

In [52]:
optimal_sequence

[4, 6, 0, 2, 3, 1, 5]

In [53]:
print(schedule(optimal_sequence,optimal_times,PROC))

|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Machine: 0 |            | Machine: 1 |            | Machine: 2 |            | Machine: 3 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Idle:    0 |            | Idle:    0 |            | Idle:    4 |            | Idle:    0 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Start:   7 |            | Start:  13 |            | Start:  19 |            | Start:  26 |
| Job:  1 | Wait:    0 | Proc:    3 | Wait:    3 | Proc:    6 | Wait:    0 | Proc:    7 | Wait:    0 | Proc:    3 |
|         |            | Stop:   10 |            | Stop:   19 |            | Stop:   26 |            | Stop:   29 |
|---------|------------|------------|------------|------------|---------

# 9. Calculate ANOVA

In [ ]:
import numpy as np
from scipy.stats import f_oneway

num_runs = 10
optimal_times_list = []
for run in range(num_runs):
    print(f"Run {run+1}")
    optimal_sequence = GA_sequence(PROC)
    optimal_times_list.append(stop_time[-1][-1])

In [ ]:
f_val, p_val = f_oneway(*[optimal_times_list for _ in range(num_runs)])

In [ ]:
print("\nANOVA results:")
print(f"F-value: {f_val}")
print(f"P-value: {p_val}")